In [7]:
import pandas as pd
import math
import numpy as np
import pprint


data=pd.read_csv("Car_Dataset.csv")
print("\n Input Data Set is:\n", data)
features = [f for f in data]
# print(type(features))
features.remove("german_made")


class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

def find_entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["Answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

def info_gain(examples, attr):
    uniq = np.unique(examples[attr])
    gain = find_entropy(examples)
    for u in uniq:
        subdata = examples[examples[attr] == u]
        sub_e = find_entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
    return gain


def id3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    uniq = np.unique(examples[max_feat])
    for u in uniq:
        subdata = examples[examples[max_feat] == u]
        if find_entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["german_made"])
            root.children.append(newNode)
        else:
            tempNode = Node()
            tempNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = id3(subdata, new_attrs)
            tempNode.children.append(child)
            root.children.append(tempNode)
    return root


def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" : ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

root = id3(data, features)
print("Final decision tree:\n")
printTree(root)


 Input Data Set is:
     Origin Manufacturer   color  year       model german_made
0  Germany   Volkswagen    blue  2017      Golf R           Y
1  Germany          BMW   black  2018          X7           Y
2      USA         Ford     red  2019  Mustang GT           N
3    Japan       Toyota  silver  2017       Camry           N
4  Germany   Volkswagen    grey  2013      Toureg           Y


KeyError: 'Answer'